In [1]:
import cv2
import PIL
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import tqdm
import os

from PIL import Image
from torchvision import transforms as T
from torchvision.models.alexnet import AlexNet
from torchvision.models.resnet import resnet18, resnet34, resnet50, resnet101, resnet152
from torch.utils.data import IterableDataset, DataLoader, Dataset
from sklearn.metrics import classification_report, accuracy_score, f1_score, average_precision_score
from sklearn.model_selection import train_test_split, KFold

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


def linear_combination(x, y, epsilon): 
    return epsilon*x + (1-epsilon)*y


def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss


class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon:float=0.1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
    
    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return linear_combination(loss/n, nll, self.epsilon)
    
    
class BonzDataset(Dataset):
    def __init__(self, data):
        super(BonzDataset, self).__init__()
        self.image_prefixes = data.image_prefixes.values
        self.features = data.features.values
        self.img_tensors = data.img_tensors.values
        if 'labels' in data:
            self.labels = data.labels.values
            self.one_hot_labels = data.one_hot_labels.values
        else:
            self.labels=None
    
    def __len__(self):
        return len(self.image_prefixes)
    
    def __getitem__(self, idx):
        outputs = (self.img_tensors[idx], self.features[idx],)
        if self.labels is not None:
            outputs += (self.one_hot_labels[idx], self.labels[idx])
        return outputs
    
        
class Bonz(nn.Module):
    def __init__(self, hidden_dim=100, feature_selection=12):
        super(Bonz, self).__init__()
        self.resnet = resnet50(True)
        self.dropout1 = nn.Dropout(0.1)
        self.BiLSTM = nn.LSTM(2048, hidden_dim, num_layers=2, batch_first=True, bidirectional=True, dropout=0)
        self.bn = nn.BatchNorm1d(hidden_dim*2 + feature_selection)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim*2 + feature_selection, hidden_dim*2 + feature_selection),
            nn.LeakyReLU(0.001, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim*2 + feature_selection, hidden_dim*2),
            nn.LeakyReLU(0.001, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim*2, 20)
        )
        #self.classifier.apply(self._init_weights)
        self.classifier.apply(self._xavier)
        self.BiLSTM.apply(self._xavier)
    
    def forward(self, imgs, features=None):
        # Generate featuress from each images
        x = []
        for img in imgs:
            temp = self.do_resnet(img)
            x.append(temp.unsqueeze(1))
        x = torch.cat(x, 1)
        x = self.dropout1(x)
        
        # LSTM step
        x, _ = self.BiLSTM(x)
        lstm_features = x[:,-1,:]
        
        # Concate features
        total_features = torch.cat([lstm_features, features], -1)
        total_features = self.bn(total_features)
        predict = self.classifier(total_features)

        return (predict, lstm_features)
    
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding, nn.Conv1d)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, (nn.Linear, nn.Conv1d)) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
            
    def _xavier(self, module):
        for name, param in module.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)
            elif 'bias' in name:
                param.data.zero_()
    
    def do_resnet(self, img):
        x = self.resnet.conv1(img)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        
        return x
    
    def freeze_resnet(self):
        for w in self.resnet.parameters():
            w.requires_grad = False
            
    def unfreeze_resnet(self):
        for w in self.resnet.parameters():
            w.requires_grad = True 

In [3]:
model = Bonz()


In [73]:
data = pd.DataFrame({'a': [0,1,2]*5, 'b': [0,1,2]*5})
data.a.values

array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2], dtype=int64)

In [2]:
def get_data(path):
    data = pd.read_csv(path)
    resnet_idx = list(data.columns).index('data_AUTOGRAPHER_RESNET_mean_tench, Tinca tinca')
    new_data = data.iloc[:,:resnet_idx]
    new_data = new_data.drop([col_len for col_len in new_data.keys() if '_len' in col_len], 1) # Drop columns with _LEN
    new_data['labels'] = [int(i[-2:])-1 for i in new_data.event_id.values]
    new_data['image_prefixes'] = list(map(lambda x,y,z: str(x)+'_'+str(y)+'_'+str(z), 
                                      new_data.sub_id.values, 
                                      new_data.source.values, 
                                      new_data.event_id.values))
    new_data['one_hot_labels'] = list(map(lambda x: nn.functional.one_hot(torch.tensor(x), 20).float(), new_data.labels.values))
    new_data['features'] = [torch.tensor(i).float() for i in new_data.iloc[:, 3:-3].values]
    new_data['img_tensors'] = get_img_tensors(new_data['image_prefixes'].values)
    return new_data.iloc[:, -5:]


def get_test_data(path):
    data = pd.read_csv(path, index_col=0)
    resnet_idx = list(data.columns).index('data_AUTOGRAPHER_RESNET_mean_tench, Tinca tinca')
    new_data = data.iloc[:,:resnet_idx]
    new_data = new_data.drop([col_len for col_len in new_data.keys() if '_len' in col_len], 1) # Drop columns with _LEN
    new_data['image_prefixes'] = list(map(lambda x,y: str(x)+'_pred'+str(y), 
                                      new_data.sub_id.values, 
                                      new_data.event_id.values))
    new_data['features'] = [torch.tensor(i).float() for i in new_data.iloc[:, 3:-1].values]
    new_data['img_tensors'] = get_img_tensors(new_data['image_prefixes'].values)
    return new_data.iloc[:, [0,1,2,-3,-2,-1]]


def get_img_tensors(image_prefixes):
    transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    temp = []
    
    for img_prefix in tqdm.notebook.tqdm(image_prefixes):
        img_paths = []
        i = 0
        img_name = img_prefix+'_'+str(i)+'.jpg'
        while img_name in os.listdir('./OUTPUT_MERGED/AUTOGRAPHER/'):
            img_paths.append('./OUTPUT_MERGED/AUTOGRAPHER/'+img_name)
            i += 1
            img_name = img_prefix+'_'+str(i)+'.jpg'

        # Transform images to tensors
        img_tensors = []
        for path in img_paths:
            img = Image.open(path)
            img_tensors.append(transform(img))
            
        # padding img tensors
        if len(img_tensors) < 16:
            dump = torch.zeros((3,224,224)).float()
            dump = [dump] * (16 - len(img_tensors))
            img_tensors.extend(dump)
            
        temp.append(img_tensors)
    
    return temp

def check_params(model):
    model.freeze_resnet()
    print(sum([i.numel() for i in model.parameters() if i.requires_grad]))
    model.unfreeze_resnet()
    print(sum([i.numel() for i in model.parameters() if i.requires_grad]))
    

        
train_data = get_data('train_min_max.csv')
#train_data = train_data.sort_values(by='image_prefixes', ignore_index=True)
test_data = get_test_data('test_min_max.csv')


model = Bonz(hidden_dim=1024, feature_selection=train_data.features[0].shape[0])
model.to(DEVICE)
torch.save(model.state_dict(), 'origin_sd.pt')
origin_state_dict = torch.load('origin_sd.pt')

check_params(model)


59363526
84920558


In [23]:
import math

model.freeze_resnet()
model.load_state_dict(origin_state_dict)

start_lr = 1e-6
end_lr = 0.1
lr_find_epochs = 10

dataloader = DataLoader(BonzDataset(train_data), batch_size=40)

optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), start_lr)
criterion = nn.CrossEntropyLoss()

lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (lr_find_epochs * len(dataloader)))
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# Make lists to capture the logs

lr_find_loss = []
lr_find_lr = []

iter = 0

smoothing = 0.05

for i in range(lr_find_epochs):
    for img_tensors, features, one_hot_label, label in dataloader:

        # Send to device
        img_tensors = [ts.to(DEVICE) for ts in img_tensors]
        features = features.to(DEVICE)
        one_hot_label = one_hot_label.to(DEVICE)
        label = label.to(DEVICE)

        # Training mode and zero gradients
        model.train()
        optimizer.zero_grad()

        # Get outputs to calc loss
        outputs = model(img_tensors, features)
        loss = criterion(outputs, label)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update LR
        scheduler.step()
        lr_step = optimizer.state_dict()["param_groups"][0]["lr"]
        lr_find_lr.append(lr_step)

        # smooth the loss
        if iter==0:
            lr_find_loss.append(loss.item())
        else:
            loss = smoothing  * loss + (1 - smoothing) * lr_find_loss[-1]
            lr_find_loss.append(loss.item())

        iter += 1

for a, b in zip(lr_find_lr, lr_find_loss):
    print(f'{a}\t{b}')

1.1787686347935873e-06	3.0017285346984863
1.3894954943731377e-06	3.0015506744384766
1.637893706954064e-06	3.0012686252593994
1.93069772888325e-06	3.0019760131835938
2.275845926074788e-06	3.0035910606384277
2.6826957952797253e-06	3.00378680229187
3.162277660168379e-06	3.0037596225738525
3.72759372031494e-06	3.0027661323547363
4.393970560760791e-06	3.0021867752075195
5.1794746792312115e-06	3.0006489753723145
6.1054022965853284e-06	3.0004799365997314
7.19685673001152e-06	3.0001814365386963
8.48342898244072e-06	2.999457359313965
1e-05	3.0001561641693115
1.1787686347935872e-05	2.999746561050415
1.3894954943731378e-05	3.000974178314209
1.6378937069540644e-05	3.0009868144989014
1.93069772888325e-05	3.001993179321289
2.2758459260747882e-05	3.0037615299224854
2.682695795279726e-05	3.0045859813690186
3.162277660168379e-05	3.004239082336426
3.72759372031494e-05	3.004592180252075
4.39397056076079e-05	3.0039453506469727
5.1794746792312105e-05	3.00384521484375
6.105402296585325e-05	3.004593133926391

In [3]:
model.freeze_resnet()
model.load_state_dict(origin_state_dict)

start_lr = 1e-6
end_lr = 0.1
lr_find_epochs = 20

dataloader = DataLoader(BonzDataset(train_data), batch_size=BATCH_SIZE, shuffle=True)

optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), start_lr)
criterion = nn.CrossEntropyLoss()

scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 
                                              base_lr=start_lr, 
                                              max_lr=end_lr, 
                                              step_size_up=lr_find_epochs)

# Make lists to capture the logs

lr_find_acc = []
lr_find_lr = []

for i in range(lr_find_epochs):
    
    predicts = []
    y_true = []
    
    for img_tensors, features, one_hot_label, label in dataloader:

        img_tensors = [ts.to(DEVICE) for ts in img_tensors]
        features = features.to(DEVICE)
        one_hot_label = one_hot_label.to(DEVICE)
        label = label.to(DEVICE)

        model.train()
        optimizer.zero_grad()

        predict = model(img_tensors, features)
        loss = criterion(predict, label)

        loss.backward()
        optimizer.step()
        
        predict = predict.detach().cpu()
        predict = torch.argmax(predict, 1)
        predicts.extend(predict.tolist())
        y_true.extend(label.detach().cpu().tolist())
    
    train_acc = accuracy_score(y_true, predicts)
    print(f'epoch={i}, Acc={train_acc}')
    
    lr_step = optimizer.state_dict()["param_groups"][0]["lr"]
    lr_find_lr.append(lr_step)
    lr_find_acc.append(train_acc)

    scheduler.step()
        
        
        
for a, b in zip(lr_find_lr, lr_find_acc):
    print(f'{a}\t{b}')

NameError: name 'BATCH_SIZE' is not defined

In [3]:
MAX_LR = 0.09
MIN_LR = 0.02
NUM_EPOCH = 40
BATCH_SIZE = 32
DRY_RUN = 0


# Split data
'''
train_dataset = train_data.loc[140:]
val_dataset = train_data.loc[:139]
'''
#train_dataset, val_dataset = train_test_split(train_data, test_size=0.2)

train_dataset, val_dataset = train_data, train_data

train_dataset = BonzDataset(train_dataset)
val_dataset = BonzDataset(val_dataset)

data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

best_metrics = {'train_loss': 1e10, 
                'val_loss': 1e10, 
                'train_acc': 0, 
                'val_acc': 0}

metrics = {'train_loss': [], 
           'val_loss': [], 
           'train_acc': [], 
           'val_acc': []}

''' TRAINING WITH FREEZE ResNet
'''
model.freeze_resnet()
model.load_state_dict(origin_state_dict) #Load origin state dict

loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, betas=(0.9, 0.99))
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=MAX_LR)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=10, verbose=True)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 
                                              base_lr=MIN_LR, 
                                              max_lr=MAX_LR, 
                                              step_size_up=(NUM_EPOCH//3)*len(data_loader))

for e in tqdm.notebook.trange(NUM_EPOCH):
    
    # Training Phase
    model.train()
    s = time.time()
    train_loss = 0
    predicts = []
    y_true = []
    
    for img_tensors, features, one_hot_label, label in data_loader:
        img_tensors = [ts.to(DEVICE) for ts in img_tensors]
        features = features.to(DEVICE)
        one_hot_label = one_hot_label.to(DEVICE)
        label = label.to(DEVICE)

        optimizer.zero_grad()
        predict = model(img_tensors, features)[0]
        loss = loss_fn(predict, label)
        loss.backward()
        optimizer.step()
        if e >= DRY_RUN:
            scheduler.step()

        train_loss += loss.item()
        predict = predict.detach().cpu()
        predict = torch.argmax(predict, 1)
        predicts.extend(predict.tolist())
        y_true.extend(label.detach().cpu().tolist())

    train_acc = accuracy_score(y_true, predicts)
    lr_step = optimizer.state_dict()["param_groups"][0]["lr"]

    # Validation Phase
    model.eval()
    val_loss = 0
    predicts = []
    y_true = []
    lstm_features = []
    
    for img_tensors, features, one_hot_label, label in val_data_loader:
        img_tensors = [ts.to(DEVICE) for ts in img_tensors]
        features = features.to(DEVICE)
        one_hot_label = one_hot_label.to(DEVICE)
        label = label.to(DEVICE)

        with torch.no_grad():
            predict, lstm = model(img_tensors, features)
            loss = loss_fn(predict, label)
            val_loss += loss.item()
        predict = predict.detach().cpu()
        predict = torch.argmax(predict, 1)
        predicts.extend(predict.tolist())
        y_true.extend(label.detach().cpu().tolist())
        lstm_features.extend(lstm.detach().cpu().tolist())

    val_acc = accuracy_score(y_true, predicts)

    # Scheduler step wise
    #scheduler.step(0.25*train_loss + 0.75*val_loss)
    
    print(f'Epoch={e}, Train_loss={train_loss:.1f}, Val_loss={val_loss:.1f}, Train_acc={train_acc*100:.2f}, Val_acc={val_acc*100:.2f}, LR={lr_step:.2e}, , Time={time.time()-s:.2f}')
    
    # Store metrics
    metrics['train_acc'].append(train_acc)
    metrics['val_acc'].append(val_acc)
    metrics['train_loss'].append(train_loss)
    metrics['val_loss'].append(val_loss)
    
    # Save best model
    if val_loss < best_metrics['val_loss']:
        best_metrics['train_acc'] = train_acc
        best_metrics['val_acc'] = val_acc
        best_metrics['train_loss'] = train_loss
        best_metrics['val_loss'] = val_loss

        torch.save(model.state_dict(), 'best_model.pt')
        print(f'BEST MODEL at Epoch = {e}')

Epoch=0, Train_loss=30.7, Val_loss=26.6, Train_acc=7.14, Val_acc=6.43, LR=2.54e-02, , Time=17.84
BEST MODEL at Epoch = 0
Epoch=1, Train_loss=28.7, Val_loss=29.7, Train_acc=15.36, Val_acc=5.00, LR=3.08e-02, , Time=17.58
Epoch=2, Train_loss=25.8, Val_loss=22.2, Train_acc=18.93, Val_acc=19.29, LR=3.62e-02, , Time=17.63
BEST MODEL at Epoch = 2
Epoch=3, Train_loss=22.0, Val_loss=18.1, Train_acc=24.64, Val_acc=46.43, LR=4.15e-02, , Time=17.78
BEST MODEL at Epoch = 3
Epoch=4, Train_loss=19.3, Val_loss=30.0, Train_acc=31.07, Val_acc=28.57, LR=4.69e-02, , Time=18.10
Epoch=5, Train_loss=19.2, Val_loss=36.0, Train_acc=35.36, Val_acc=24.64, LR=5.23e-02, , Time=18.15
Epoch=6, Train_loss=17.4, Val_loss=36.0, Train_acc=38.93, Val_acc=20.00, LR=5.77e-02, , Time=18.17
Epoch=7, Train_loss=14.9, Val_loss=77.2, Train_acc=50.71, Val_acc=6.07, LR=6.31e-02, , Time=18.31
Epoch=8, Train_loss=13.7, Val_loss=63.5, Train_acc=52.50, Val_acc=11.07, LR=6.85e-02, , Time=18.37
Epoch=9, Train_loss=10.7, Val_loss=73.5, 

In [69]:
''' Training with UNFREEZE ResNet
''' 
model.unfreeze_resnet()
#optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, betas=(0.9, 0.99))
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, momentum=0.9, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

for e in tqdm.notebook.trange(50):
    model.train()
    model.zero_grad()
    s = time.time()
    train_loss = 0
    predicts = []
    y_true = []
    data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    for img_tensors, features, one_hot_label, label in data_loader:
        img_tensors = [ts.to(DEVICE) for ts in img_tensors]
        features = features.to(DEVICE)
        one_hot_label = one_hot_label.to(DEVICE)
        label = label.to(DEVICE)

        optimizer.zero_grad()
        predict = model(img_tensors, features)
        loss = loss_fn(predict, label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predict = predict.detach().cpu()
        predict = torch.argmax(predict, 1)
        predicts.extend(predict.tolist())
        y_true.extend(label.detach().cpu().tolist())

    train_acc = accuracy_score(y_true, predicts)

    # Validate data
    model.eval()
    predicts = []
    y_true = []
    val_loss = 0
    val_data_loader = DataLoader(val_dataset, batch_size=4)
    for img_tensors, features, one_hot_label, label in val_data_loader:
        img_tensors = [ts.to(DEVICE) for ts in img_tensors]
        features = features.to(DEVICE)
        one_hot_label = one_hot_label.to(DEVICE)
        label = label.to(DEVICE)

        with torch.no_grad():
            predict = model(img_tensors, features)
            loss = loss_fn(predict, label)
            val_loss += loss.item()
        predict = predict.detach().cpu()
        predict = torch.argmax(predict, 1)
        predicts.extend(predict.tolist())
        y_true.extend(label.detach().cpu().tolist())

    val_acc = accuracy_score(y_true, predicts)

    # Scheduler step wise
    scheduler.step(0.1*train_loss + 0.9*val_loss)

    if (0.1*train_loss + 0.9*val_loss) <= (0.1*best_metrics['train_loss'] + 0.9*best_metrics['val_loss']):
        if (0.1*train_acc + 0.9*val_acc) > (0.1*best_metrics['train_acc'] + 0.9*best_metrics['val_acc']):
            best_metrics['train_acc'] = train_acc
            best_metrics['val_acc'] = val_acc
            best_metrics['train_loss'] = train_loss
            best_metrics['val_loss'] = val_loss

            torch.save(model.state_dict(), 'best_model.pt')
            print(f'BEST MODEL at Epoch = {e}: Train_loss={train_loss:.1f}, Val_loss={val_loss:.1f}, Train_acc={train_acc:.2f}, Val_acc={val_acc:.2f}, Time={time.time()-s:.2f}')

BEST MODEL at Epoch = 0: Train_loss=27.9, Val_loss=27.0, Train_acc=0.04, Val_acc=0.05, Time=19.37
BEST MODEL at Epoch = 2: Train_loss=27.5, Val_loss=26.6, Train_acc=0.05, Val_acc=0.05, Time=19.57
BEST MODEL at Epoch = 3: Train_loss=27.5, Val_loss=26.0, Train_acc=0.04, Val_acc=0.08, Time=19.52
BEST MODEL at Epoch = 4: Train_loss=25.5, Val_loss=23.1, Train_acc=0.09, Val_acc=0.14, Time=19.57
BEST MODEL at Epoch = 8: Train_loss=23.9, Val_loss=22.3, Train_acc=0.11, Val_acc=0.21, Time=20.09
BEST MODEL at Epoch = 10: Train_loss=21.9, Val_loss=20.2, Train_acc=0.12, Val_acc=0.23, Time=20.01
BEST MODEL at Epoch = 11: Train_loss=20.0, Val_loss=18.6, Train_acc=0.20, Val_acc=0.29, Time=20.05
BEST MODEL at Epoch = 13: Train_loss=18.9, Val_loss=17.1, Train_acc=0.26, Val_acc=0.30, Time=20.07
BEST MODEL at Epoch = 14: Train_loss=18.3, Val_loss=15.6, Train_acc=0.24, Val_acc=0.36, Time=20.25
BEST MODEL at Epoch = 18: Train_loss=17.2, Val_loss=14.7, Train_acc=0.32, Val_acc=0.39, Time=19.98
BEST MODEL at E

Epoch    10: reducing learning rate of group 0 to 5.0000e-05.
Epoch    21: reducing learning rate of group 0 to 2.5000e-05.
Epoch    27: reducing learning rate of group 0 to 1.2500e-05.
Epoch    34: reducing learning rate of group 0 to 6.2500e-06.
Epoch    40: reducing learning rate of group 0 to 3.1250e-06.
Epoch    46: reducing learning rate of group 0 to 1.5625e-06.



In [5]:
model.eval()
model.load_state_dict(torch.load('best_model.pt'))

predict_matrix = []
lstm_features = []
val_loss = 0
val_data_loader = DataLoader(BonzDataset(train_data), batch_size=BATCH_SIZE)

for img_tensors, features, one_hot_label, label in val_data_loader:
    img_tensors = [ts.to(DEVICE) for ts in img_tensors]
    features = features.to(DEVICE)
    one_hot_label = one_hot_label.to(DEVICE)
    label = label.to(DEVICE)
    
    with torch.no_grad():
        predict, lstm = model(img_tensors, features)
        
    predict = predict.detach().cpu()
    predict_matrix.extend(torch.softmax(predict, 1))
    lstm_features.extend(lstm.detach().cpu().tolist())
    
onehot_true = np.array(list(i.numpy() for i in train_data.one_hot_labels.values))
onehot_predict = torch.stack(predict_matrix).numpy()

average_precision_score(onehot_true, onehot_predict)

0.9951127895797012

In [8]:
train_features = pd.DataFrame(lstm_features)
train_features.to_csv('train_features.csv', index=False)

In [9]:
model.eval()

predict_matrix = []
lstm_features = []
val_data_loader = DataLoader(BonzDataset(test_data), batch_size=BATCH_SIZE)

for img_tensors, features in val_data_loader:
    img_tensors = [ts.to(DEVICE) for ts in img_tensors]
    features = features.to(DEVICE)
    with torch.no_grad():
        predict, lstm = model(img_tensors, features)
    predict = predict.detach().cpu()
    predict_matrix.extend(torch.softmax(predict, 1))
    lstm_features.extend(lstm.detach().cpu().tolist())
    
test_features = pd.DataFrame(lstm_features)
test_features.to_csv('test_features.csv', index=False)

onehot_predict = torch.stack(predict_matrix).numpy()

COLUMN_NAMES = ['act'+str(i)+str(j) for i in range(2) for j in range(10)]
COLUMN_NAMES.pop(0)
COLUMN_NAMES.append('act20')

df_prob = pd.DataFrame(data=onehot_predict, columns=COLUMN_NAMES)
df_prob['event'] = test_data['image_prefixes']

submission = []
for act in list(df_prob.keys())[:-1]:
    ranked_ = df_prob.sort_values(by=[act], ascending=False)['event'].values
    ranked_ = act+' '+ranked_
    submission.extend(ranked_)

pd.DataFrame(submission).to_csv('submission.txt', header=False, index=False)

np.argmax(onehot_predict, 1)

array([ 7,  0,  7, 14,  7, 18, 18, 11,  7,  7, 13, 12, 14, 13, 12, 14, 11,
       13, 14, 14, 13, 12, 13, 14, 13, 13, 13, 11, 12, 13, 13, 13,  7,  7,
       13, 12, 14, 18, 13, 13, 13, 18, 13, 13, 14, 12, 13, 13, 11, 13, 14,
       14, 13, 13, 13, 13,  7, 13, 13, 11,  7, 13, 14, 13, 13, 12, 13, 18,
       11, 12, 13, 13, 13, 13, 14,  7, 13, 12,  7, 13, 12, 12, 11, 19, 11,
       12, 13, 11,  7, 13, 13, 18, 13, 11, 12, 12, 13, 14,  7,  7, 13, 12,
       13,  7, 12, 12, 13, 13, 12, 13, 13, 13, 13, 13, 18,  7, 13, 16, 13,
       13, 11, 13, 14,  7, 18, 12, 14, 13, 13, 13, 13,  7, 13,  7, 12, 13,
       13, 13, 12, 13], dtype=int64)